# dMRI Data Reconstruction


In this notebook, we will reconstruct MRI imgaes from raw data by using Python.This includes: 1. Data processing; 2. DTI reconstruction and 3. DKI reocnstruction.

## Data Preprocessing

Data preprocessing is quit important for dMRI reconstruction. Different data preprocessing may lead to different reconstruction image qualities, which will make the comparation of different reconstruct methods unreliable. Thus, here we first preprocessing MRI by following same steps: denosing, topup (susceptibility-induced distortion correction) and eddy current-induced distortion and motion correction.

### __Import python libraries__

In [ ]:
import os #TO control directories
import nibabel as nib # read and save medical images

import numpy as np
from dipy.denoise.localpca import mppca #denoising
import nipype.interfaces.fsl as fsl #topup
from nipype.interfaces.fsl import TOPUP
from nipype.interfaces.fsl import ApplyTOPUP
from nipype.interfaces.fsl import Eddy
from nipype.testing import anatfile

import timeit #compute time, useage: timeit.timeit()

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
import plotly
import plotly.express as px
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

import warnings

### Set data path and load data

In [ ]:
data_path = "/home/erjun/githubEZ/dMRI_BHS/dMRI_data/dwi"
ap_file = 'sub-032216_ses-001_dir-AP_run-1_dwi.nii.gz' # dMRI data
pa_file = 'sub-032216_ses-001_dir-PA_run-1_dwi.nii.gz' 
bvals_file = 'sub-032216_ses-001_dir-AP_run-1_dwi.bval' # bval file
bvecs_file = 'sub-032216_ses-001_dir-AP_run-1_dwi.bvec' # bvec file
denoised_file = 'sub-032216_ses-001_dir-AP_run-1_dwi_denoised.nii.gz' # output file after denoising

cwdir = os.getcwd()
os.chdir(data_path) #directory setting
# Load images
img = nib.load(os.path.join(data_path,ap_file)) # path.join connects data path given by
data = img.get_data()# I,age data array but without position information; need to be combine with image.affine (data position info)
# Use dipy to denoise

### Data type (NII.GZ)

- NII (NifTI, format by Neuroimaging Informatics Technology Initiative ) file is commonly used format for multi-dimensional (can be up to 7-dimensional) neuroimaging data. Fisrt four dimension: spatial dimensions and time. 

- GZ means gzip-compressed NII files.
* nib.nifil.Nifti1Image: three parts included in, namely, image data array, an affine array and image metadata.
* image metadata: machine info., voxel size and slices

Thus, in order to know the exact position of each voxel, we have to combine image data array and affine array. For more information, please check [fMRI Processing based on python](https://ff120.github.io/2016/06/12/%E8%AE%A4%E7%9F%A5%E7%A5%9E%E7%BB%8F%E7%A7%91%E5%AD%A6%E4%B8%93%E9%A2%98/%E4%BD%BF%E7%94%A8Python%E5%A4%84%E7%90%86fMRI%E6%95%B0%E6%8D%AE/).

### __Denoising__

Here we use [Marcenko-Pastur PCA algorithm](https://dipy.org/documentation/1.0.0./examples_built/denoise_mppca/) to denoise images. This algorithm has been shown to provide an optimal compromise between noise suppression and loss of anatomical information for different techniques such as DTI.

During the denoising, mppca use a 3D sliding window (decised by denoising radius, pathc_radius) to denoise. Basicaly, this 3D sliding window voxles should be larger than DTI volumes.

In [ ]:
denoised = mppca(data, patch_radius=4) # If volume is about 67, pathc_radius can be set to 2
# Save data 
nib.save(nib.Nifti1Image(data, img.affine), os.path.join(data_path,denoised_file))

print('DONE')

### TOPUP

In [ ]:
# Set default output type and test ExtractROI tool for Define b_0 image extraction function
fsl.FSLCommand.set_default_output_type('NIFTI_GZ')
# This is used to test fsl.ExtractROI, if you see bar.nii.gz file in the data_path, it works well.
fslroi = fsl.ExtractROI(in_file=anatfile, roi_file='bar.nii.gz', t_min=0,t_size=1)
fslroi.cmdline == 'fslroi %s bar.nii.gz 0 1' % anatfile

In [ ]:
# Define b_0 image extraction function

def Extract_b0_Image(input_Image, output_Image):
    "To run this, please first make sure you install fsl and can run it"
    "One method is that run fsl and thi pre-processing code in the same terminal"
    fslroi = fsl.ExtractROI(in_file=input_Image,roi_file=output_Image,t_min=0, t_size=1)
    fslroi.run()
    
# Test
#extract_b0(ap_file, 'bar.nii.gz')
#fslroi.cmdline == 'fslroi %s bar.nii.gz 0 1' % ap_file

In [ ]:
# Extract b0 images
Extract_b0_Image(ap_file, 'epi_b0.nii.gz')
Extract_b0_Image(pa_file, 'epi_rev_b0.nii.gz')

In [ ]:
def nib_rd_img(path):
    image_data = nib.load(path).get_data()
    return image_data
def nib_show_img(img0):
    img00=img0[:,:,30]
    zmax0=img00.max()
    fig=px.imshow(img0[:,:,30],color_continuous_scale="Viridis",zmin=0,zmax=10000,x=None, y=None,labels={},template="plotly_white")
    fig.show()

In [ ]:
data22 = nib_rd_img('epi_b0.nii.gz')
nib_show_img(data22)

In [ ]:
# Use fslmerge to concatenate images
merger = fsl.Merge()
merger.inputs.in_files = ['epi_b0.nii.gz','epi_rev_b0.nii.gz']
merger.inputs.dimension = 't'
merger.inputs.output_type = 'NIFTI_GZ'
#merger = fsl.Merge(in_files=['epi_b0.nii.gz','epi_rev_b0.nii.gz'],dimension = 't',output_type='NIFTI_GZ')
merger.run()

file = open('topup_encoding.txt','w')
file.write('0 1 0 0.05\n0 -1 0 0.05')
file.close()

In [ ]:
topup = TOPUP()
topup.inputs.in_file = 'epi_b0_merged.nii.gz'
topup.inputs.encoding_file = "topup_encoding.txt"
topup.inputs.output_type = 'NIFTI_GZ'
topup.cmdline
topup.run()

In [ ]:
#------------------------------------------------
#            FSL ApplyTOPUP 
#------------------------------------------------
applytopup = ApplyTOPUP()
applytopup.inputs.in_files = ['epi_b0.nii.gz', 'epi_rev_b0.nii.gz']
applytopup.inputs.encoding_file = "topup_encoding.txt"
applytopup.inputs.in_topup_fieldcoef = "epi_b0_merged_base_fieldcoef.nii.gz"
applytopup.inputs.in_topup_movpar = "epi_b0_merged_base_movpar.txt"
applytopup.inputs.output_type = "NIFTI_GZ"
applytopup.cmdline
applytopup.run()        
print('DONE')

### EDDY

In [ ]:
btr = fsl.BET(in_file= 'epi_b0.nii.gz',#'epi_b0_corrected.nii.gz',
              frac=0.2, out_file='brain.nii.gz', mask=True)
btr.run()

# total nuber of volumes in dwi data
img = nib.load(denoised_file).get_data()
nvolumes = img.shape[-1]

file = open('index.txt','w')
for i in range(0, nvolumes):
    file.write('1 ')
file.close()

eddy = Eddy()
eddy.inputs.in_file = denoised_file
eddy.inputs.in_mask  = 'brain_mask.nii.gz'
eddy.inputs.in_index = 'index.txt'
eddy.inputs.in_acqp  = 'topup_encoding.txt'
eddy.in_topup_fieldcoef = "epi_b0_merged_base_fieldcoef.nii.gz"
eddy.inputs.in_bvec  = bvecs_file
eddy.inputs.in_bval  = bvals_file
eddy.inputs.use_cuda = False
eddy.inputs.is_shelled=True
eddy.cmdline
eddy.run() 
print('DONE')

## DTI Reconstruction

In [ ]:
import math
from skimage import io #用于读取保存或显示图片或者视频
import time

from dipy.io import read_bvals_bvecs
from dipy.core.gradients import gradient_table
from dipy.reconst.dti import TensorModel
from dipy.reconst.dti import fractional_anisotropy
from dipy.reconst.dti import color_fa
import dipy.reconst.dki as dki

In [ ]:
# Set new data path for DTI reconstruction
#data_path = "/home/erjun/githubEZ/dHCP/sub-CC00060XX03/ses-12501/dwi"
#dwi_file = 'sub-CC00060XX03_ses-12501_desc-preproc_dwi.nii.gz'
brainmask_file = 'brain_mask.nii.gz'
#bval = 'sub-CC00060XX03_ses-12501_desc-preproc_dwi.bval'
#bvec = 'sub-CC00060XX03_ses-12501_desc-preproc_dwi.bvec'
#os.chdir(data_path)


In [ ]:
# Load data files
img1 = nib.load(os.path.join(data_path,ap_file))
data = img1.get_data()

img2 = nib.load(os.path.join(data_path,brainmask_file))
brainmask = img2.get_data()

bvals, bvecs = read_bvals_bvecs(os.path.join(bvals_file),
                                os.path.join(data_path,bvecs_file))
gtab = gradient_table(bvals, bvecs)

In [ ]:
# DTI model
ten_model = TensorModel(gtab)
ten_fit = ten_model.fit(data, brainmask)
        
# Save DTI parametric maps
if not os.path.exists(data_path+'/DTI/'):
    os.mkdir(data_path+'/DTI')
output_path = data_path+'/DTI/'
        
DTI_FA = ten_fit.fa
DTI_AD = ten_fit.ad
DTI_RD = ten_fit.rd
DTI_MD = ten_fit.md
        
nib.save(nib.Nifti1Image(DTI_FA, img1.affine), os.path.join(output_path,'FA.nii.gz'))
nib.save(nib.Nifti1Image(DTI_MD, img1.affine), os.path.join(output_path,'MD.nii.gz'))
nib.save(nib.Nifti1Image(DTI_RD, img1.affine), os.path.join(output_path,'RD.nii.gz'))
nib.save(nib.Nifti1Image(DTI_AD, img1.affine), os.path.join(output_path,'AD.nii.gz'))
    
#Save FA RGB map
fa = fractional_anisotropy(ten_fit.evals)
cfa = color_fa(fa, ten_fit.evecs)
DTI_FA = np.clip(fa, 0, 1)
DTI_RGB = color_fa(fa, ten_fit.evecs)

nib.save(nib.Nifti1Image(np.array(255 * cfa, 'uint8'), img1.affine), os.path.join(output_path,'FA_RGB.nii.gz'))

print('Done!')

In [ ]:
# DKI MODEL
dkimodel = dki.DiffusionKurtosisModel(gtab)
dkifit = dkimodel.fit(data, brainmask)
        
# Save DKI parametric maps
if not os.path.exists(data_path+'/DKI/'):
    os.mkdir(data_path+'/DKI')
data_path_saveImage = data_path+'/DKI/'
        
DKI_FA = dkifit.fa
DKI_MD = dkifit.md
DKI_RD = dkifit.rd
DKI_AD = dkifit.ad

DKI_MK = dkifit.mk(0, 3)
DKI_AK = dkifit.ak(0, 3)
DKI_RK = dkifit.rk(0, 3)
        
nib.save(nib.Nifti1Image(DKI_FA, img1.affine), os.path.join(data_path_saveImage,'dki_FA.nii.gz'))
nib.save(nib.Nifti1Image(DKI_MD, img1.affine), os.path.join(data_path_saveImage,'dki_MD.nii.gz'))
nib.save(nib.Nifti1Image(DKI_RD, img1.affine), os.path.join(data_path_saveImage,'dki_RD.nii.gz'))
nib.save(nib.Nifti1Image(DKI_AD, img1.affine), os.path.join(data_path_saveImage,'dki_AD.nii.gz'))
        
nib.save(nib.Nifti1Image(DKI_AK, img1.affine), os.path.join(data_path_saveImage,'AK.nii.gz'))
nib.save(nib.Nifti1Image(DKI_RK, img1.affine), os.path.join(data_path_saveImage,'RK.nii.gz'))
nib.save(nib.Nifti1Image(DKI_MK, img1.affine), os.path.join(data_path_saveImage,'MK.nii.gz'))
        
print('DONE!')

## Visualization

In this section, I first show the basic images generate during preprocessing and final image reconstruction process. Then I will go the data visualization part. And before that, to create images more eassily, I will definie several image showing function first.

All the images are generated from this project and base on the data preprocessing and iamge reconstruction part.

Let's go to check what basic images we aready have!

### Basic Images

In [ ]:
# DTI Images
# set plot background
#plt.style.use('seaborn-dark')
plt.style.context('dark_background')
# plot paramter maps          
fig, [ax0, ax2, ax3, ax4] = plt.subplots(1,4,figsize=(10,8),subplot_kw={'xticks': [], 'yticks': []})
ax0.imshow(DTI_RGB[:,:,30,:]); ax0.set_title('Color coded FA',fontweight='bold',size=10)
#ax1.imshow(DTI_FA[:,30,:]); ax1.set_title('Fractional anisotropy',fontweight='bold',size=10)
ax2.imshow(DTI_MD[:,:,30]); ax2.set_title('Mean diffusivity',fontweight='bold',size=10)
ax3.imshow(DTI_RD[:,:,30]); ax3.set_title('Radial diffusivity',fontweight='bold',size=10)
ax4.imshow(DTI_AD[:,:,30]); ax4.set_title('Axial diffusivity',fontweight='bold',size=10)

In [ ]:
# DKI images         
fig, ([ax0, ax1, ax2],[ax3, ax4, ax5]) = plt.subplots(2,3,figsize=(10,8),subplot_kw={'xticks': [], 'yticks': []})
ax0.imshow(DKI_AD[:,:,30]); ax0.set_title('Axial diffusivity',fontweight='bold',size=10)
ax1.imshow(DKI_RD[:,:,30]); ax1.set_title('Radial diffusivity',fontweight='bold',size=10)
ax2.imshow(DKI_MD[:,:,30]); ax2.set_title('Mean diffusivity',fontweight='bold',size=10)
ax3.imshow(DKI_AK[:,:,30]); ax3.set_title('Axial kurtosis',fontweight='bold',size=10)
ax4.imshow(DKI_RK[:,:,30]); ax4.set_title('Radial kurtosis',fontweight='bold',size=10)
ax5.imshow(DKI_MK[:,:,30]); ax5.set_title('Mean kurtosis',fontweight='bold',size=10)

### Data Visualization

In [ ]:
# Function used to show images definition
#------------To read and show images from nii.gz file-------------------------------
def nib_read_img(path):
    image_data = nib.load(path).get_data()
    return image_data
def nib_show_img(img0,slices,intenseScale):
    img00=img0[:,:,slices]
    zmax0=img00.max()
    fig=px.imshow(img0[:,:,slices],color_continuous_scale="Viridis",zmin=0,zmax=zmax0*intenseScale/100,x=None, y=None,labels={},template="plotly_white")
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    fig.update_layout(coloraxis_showscale=False)
    fig.show()
#------------------To make functions above shorter-----------------------------------
def nib_rdshow_img(Images,Slices,IntenseScale,TitleImg):
    'Images: data path; Slices: which slice you want to see; IntenseScale: to percentage of the highest color strength'
    "TitleImg:image title, for example:'a test title'"
    warnings.filterwarnings('ignore')
    image_data = nib.load(Images).get_data()
    img00=image_data[:,:,Slices]
    zmax0=img00.max()
    fig=px.imshow(image_data[:,:,Slices],color_continuous_scale="Viridis",\
                  zmin=0,zmax=zmax0*IntenseScale/100,\
                  labels={},template="plotly_white")
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    fig.update_layout(coloraxis_showscale=False)
    fig.update_layout(title=TitleImg)
    fig.show()
#--------------------Define a function to show volume slices images--------------------
# Visualization of MRI volume slices
def vol_plot(x):
    "to create 3D  MRI figure with slider"
    vol = x
    colormax = vol.max()#获取最大array中的最大值，最后代表cmax
    volume = vol.T
    len(volume)
    r, c = volume[math.floor(len(volume)/2)].shape
    # Define frames
    import plotly.graph_objects as go
    nb_frames = len(volume)-1
    fig = go.Figure(frames=[go.Frame(
        data=go.Surface(
        z=(len(volume)-1 - k ) * np.ones((r, c)),
        surfacecolor=volume[len(volume)-1 - k],
        cmin=0, cmax=colormax
        ),
        name=str(k) # name the frame for the animation to behave properly
        )
        for k in range(nb_frames)])

    # Add data to be displayed before animation starts
    fig.add_trace(go.Surface(
        z=(len(volume)-1) * np.ones((r, c)),
        surfacecolor=volume[len(volume)-1],#np.flipud(volume[30]),
        colorscale='gray',
        cmin=0, cmax=colormax,
        colorbar=dict(thickness=20, ticklen=4)
        ))

    def frame_args(duration):
        return {
                "frame": {"duration": 500},# Duration can be used to change animate speed
                "mode": "immediate",
                "fromcurrent": True,
                "transition": {"duration": 500, "easing": "linear"},
            }

    sliders = [
                {
                    "pad": {"b": 10, "t": 60},
                    "len": 0.9,
                    "x": 0.1,
                    "y": 0,
                    "steps": [
                        {
                            "args": [[f.name], frame_args(0)],
                            "label": str(k),
                            "method": "animate",
                        }
                        for k, f in enumerate(fig.frames)
                    ],
                }
            ]

    # Layout
    fig.update_layout(
             title='Slices in volumetric data',
             width=600,
             height=600,
             scene=dict(
                        zaxis=dict(range=[-1, len(volume)-1], autorange=False),
                        aspectratio=dict(x=1, y=1, z=1),
                        ),
             updatemenus = [
                {
                    "buttons": [
                        {
                            "args": [None, frame_args(50)],
                            "label": "&#9654;", # play symbol
                            "method": "animate",
                        },
                        {
                            "args": [[None], frame_args(0)],
                            "label": "&#9724;", # pause symbol
                            "method": "animate",
                        },
                    ],
                    "direction": "left",
                    "pad": {"r": 10, "t": 70},
                    "type": "buttons",
                    "x": 0.1,
                    "y": 0,
                }
             ],
             sliders=sliders
    )

    fig.show()
#---------------------------Update function above------------------------------------
def interact_vol_plot(x,IntenseScale):
    "to create 3D  MRI figure with slider"
    vol = x
    colormax = IntenseScale*vol.max()#获取最大array中的最大值，最后代表cmax
    volume = vol.T
    len(volume)
    r, c = volume[math.floor(len(volume)/2)].shape
    # Define frames
    import plotly.graph_objects as go
    nb_frames = len(volume)-1
    fig = go.Figure(frames=[go.Frame(
        data=go.Surface(
        z=(len(volume)-1 - k ) * np.ones((r, c)),
        surfacecolor=volume[len(volume)-1 - k],
        cmin=0, cmax=colormax
        ),
        name=str(k) # name the frame for the animation to behave properly
        )
        for k in range(nb_frames)])

    # Add data to be displayed before animation starts
    fig.add_trace(go.Surface(
        z=(len(volume)-1) * np.ones((r, c)),
        surfacecolor=volume[len(volume)-1],#np.flipud(volume[30]),
        colorscale='gray',
        cmin=0, cmax=colormax,
        colorbar=dict(thickness=20, ticklen=4)
        ))
    def frame_args(duration):
        return {
                "frame": {"duration": 500},# Duration can be used to change animate speed
                "mode": "immediate",
                "fromcurrent": True,
                "transition": {"duration": 500, "easing": "linear"},
            }

    sliders = [
                {
                    "pad": {"b": 10, "t": 60},
                    "len": 0.9,
                    "x": 0.1,
                    "y": 0,
                    "steps": [
                        {
                            "args": [[f.name], frame_args(0)],
                            "label": str(k),
                            "method": "animate",
                        }
                        for k, f in enumerate(fig.frames)
                    ],
                }
            ]

    # Layout
    fig.update_layout(
             title='Volume Slices Image',
             width=600,
             height=600,
             scene=dict(
                        zaxis=dict(range=[-1, len(volume)-1], autorange=False),
                        aspectratio=dict(x=1, y=1, z=1),
                        ),
             updatemenus = [
                {
                    "buttons": [
                        {
                            "args": [None, frame_args(50)],
                            "label": "&#9654;", # play symbol
                            "method": "animate",
                        },
                        {
                            "args": [[None], frame_args(0)],
                            "label": "&#9724;", # pause symbol
                            "method": "animate",
                        },
                    ],
                    "direction": "left",
                    "pad": {"r": 10, "t": 70},
                    "type": "buttons",
                    "x": 0.1,
                    "y": 0,
                }
             ],
             sliders=sliders
    )
    fig.show()

### 3D volume slice image

This images will quickly check the general apperance of you images during the slice-by-slice animation. One can also rotate it and zoom in to see clearly.

Here, as an example, I just show FA map generated from DTI model. Once can change it to other maps, such as DTI_MD or DKI_FA.

In [ ]:
vol_plot(DTI_FA[:,:,:]) # or can use interact_vol_plot(DTI_MD,90) to show images

### Data visualization (Preprocessing part)

I am curious about what changed of the iamges during our data preprocessing. This interactive images can be used to reach this purpose.

One can control Slices to check different slice changes among them. You can control the level of high color scale to get higher image contrast. You can put cursor on images to get the exact color value if you want.

In [ ]:
def nib_rdshow_play(Slices,IntenseScale,NoX):
    warnings.filterwarnings('ignore')
    if(NoX==0):
        Image='epi_b0.nii.gz'
        TitleImg='Preprocess: bo image after extraction '
    elif(NoX==1):
        Image='epi_b0_merged.nii.gz'
        TitleImg='Preprocess: merge'
    elif(NoX==2):
        Image='epi_b0_merged_corrected.nii.gz'
        TitleImg='Preprocess: merge corrected'
    elif(NoX==3):
        Image='eddy_corrected.nii.gz'
        TitleImg='Preprocess: EDDY corrected'
    else:
        Image='brain.nii.gz'
        TitleImg='Preprocess: final'
    image_data = nib.load(Image).get_data()
    img00=image_data[:,:,Slices]
    zmax0=img00.max()
    fig=px.imshow(image_data[:,:,Slices],color_continuous_scale="Viridis",\
                  zmin=0,zmax=zmax0*IntenseScale/100,\
                  labels={},template="plotly_white")
    fig.update_xaxes(showticklabels=False)
    fig.update_yaxes(showticklabels=False)
    fig.update_layout(coloraxis_showscale=False)
    fig.update_layout(title=TitleImg)
    fig.show()
interactive(nib_rdshow_play,Slices=widgets.IntSlider(min=0,max=43,step=1,value=22),\
         IntenseScale=widgets.IntSlider(min=0,max=100,step=1,value=90),\
           NoX = widgets.Play(value=0,min=0,max=4,step=1,interval=1000,description="Press play",\
                                   disabled=False))

### Data visualization (Reconstruction part)

This part is used to compare images generated from DTI and DKI models. The founction is almost the same as virsualizaiotn above. But to make us better choose iamges, we use dropdown bar to replace slider animation.

In [ ]:
# Load image data file
Images= dict()
Images['DTI_FA_RGB']=data_path+'/DTI/FA_RGB.nii.gz'
Images['DTI_AD']=data_path+'/DTI/AD.nii.gz'
Images['DTI_FA']=data_path+'/DTI/FA.nii.gz'
Images['DTI_MD']=data_path+'/DTI/MD.nii.gz'
Images['DTI_RD']=data_path+'/DTI/RD.nii.gz'

Images['DKI_AD']=data_path+'/DKI/dki_AD.nii.gz'
Images['DKI_FA']=data_path+'/DKI/dki_FA.nii.gz'
Images['DKI_MD']=data_path+'/DKI/dki_MD.nii.gz'
Images['DKI_RD']=data_path+'/DKI/dki_RD.nii.gz'
Images['DKI_AK']=data_path+'/DKI/AK.nii.gz'
Images['DKI_MK']=data_path+'/DKI/MK.nii.gz'
Images['DKI_RK']=data_path+'/DKI/RK.nii.gz'



interactive(nib_rdshow_img,Images=Images,Slices=widgets.IntSlider(min=0,max=43,step=1,value=22),\
         IntenseScale=widgets.IntSlider(min=0,max=100,step=1,value=90),\
           TitleImg='Reconstructed Images of Different Models')